In [22]:
import random
#from operator import attrgetter

kolory=["♠️","♥️","♣️","♦️"]
wartości=["A","K","Q","W","10","9","8","7","6","5","4","3","2"]
pozycje=["N","E","S","W"]

#awersy
piki=['🂡', '🂢', '🂣', '🂤', '🂥', '🂦', '🂧', '🂨', '🂩', '🂪', '🂫', '🂭', '🂮']
kiery=['🂱', '🂲', '🂳', '🂴', '🂵', '🂶', '🂷', '🂸', '🂹', '🂺', '🂻', '🂽', '🂾']
karo=['🃁', '🃂', '🃃', '🃄', '🃅', '🃆', '🃇', '🃈', '🃉', '🃊', '🃋', '🃍', '🃎']
trefle=['🃑', '🃒', '🃓', '🃔', '🃕', '🃖', '🃗', '🃘', '🃙', '🃚', '🃛', '🃝', '🃞']
jokery=['🂿', '🃏']

class Karta():
    '''definicja karty do gry''' 
    def __init__(self, wartość: str = '', kolor: str = ''):
        self.kolor = kolor.strip()
        self.wartość = wartość.strip()

    def __str__(self)->str:
        return f'{self.get_label()}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}(wartość={self.wartość!r}, kolor={self.kolor!r})'
    
    def get_label(self)->str:
        if self.wartość == '' and self.kolor == '':
            return '?'
        return self.wartość + self.kolor
    
class CardsCollection(list[Karta]):
    '''definicja stosu kart'''
    def __init__(self, *args: list[Karta]|None):
        '''utworzenie stosu kart'''
        super().__init__(*args)
    
    def __str__(self):
        return f'{self.get_labels(self)}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}({[karta.__repr__() for karta in self]})'
    
    def w_stosie(self:CardsCollection)->list[str]:
        nazwy_kart:list[str]=[]
        for karta in self:
            nazwy_kart.append(karta.get_label())
        return(nazwy_kart)
    
    def liczba_kart(self)->int:
        '''liczba kart w stosie'''
        return len(self)
    
    @staticmethod
    def get_labels(stack: CardsCollection|list[Karta])->list[str]:
        '''lista nazw kart w stosie'''
        nazwy_kart:list[str]=[]
        for karta in stack:
            nazwy_kart.append(karta.get_label())
        return(nazwy_kart)
 
class Talia(CardsCollection):
    '''definicja talii kart do gry'''
    
    def __init__(self, *args: list[Karta]|None):
        '''utworzenie talii kart'''
        super().__init__(*args)
        if len(self) == 0: # jeśli nie podano kart, to tworzymy standardową talię 52 kart
            for kolor in kolory:
                for wartość in wartości:
                    self.append(Karta(wartość,kolor))
    
    def __str__(self):
        return f'{self.get_labels(self)}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}({[karta.__repr__() for karta in self]})'
    
    def shuffle(self):
        '''tasowanie talii'''
        random.shuffle(self) # używa algorytmu Fisher-Yates

    def tasuj_fisher_yates(self): # algorytm Fisher-Yates
        '''Fisher-Yates shuffle - algorytm tasowania O(n)'''
        for i in range(len(self) - 1, 0, -1):   # Iteracja od ostatniego elementu do pierwszego
            j = random.randint(0, i)            # Losowy indeks od 0 do bieżącego indeksu (i)
            self[i], self[j] = self[j], self[i] # Zamiana elementów (swap)

    def get_random_card(self)->Karta|None:
        '''losowanie karty z talii'''
        random_card: Karta|None = None
        if len(self) > 0:
            random_card = random.choice(self)
            self.remove(random_card)
        return(random_card)
    
    def get_first_card(self)->Karta|None:
        '''pobranie pierwszej karty z talii'''
        return(self.pop(0) if len(self) > 0 else None)
    
class Osoba():
    '''definicja osoby'''

    def __init__(self, imię: str=''):
        self.imię = imię
    
    def __str__(self):
        return f'{self.imię}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}(imię={self.imię!r})'
    
class Gracz(CardsCollection, Osoba):
    '''definicja grającego'''

    def __init__(self, *args: list[Karta]|None, imię: str = '', pozycja: str = ''):
        super().__init__(*args)  
        self.imię = imię
        self.pozycja = pozycja
    
    def __str__(self):
        s1 = "Brak imienia" if self.imię == "" else self.imię
        s2 = "Brak pozycji" if self.pozycja  == "" else self.pozycja
        return (f'{s1}, {s2} - {self.hand()}')
    
    def __repr__(self):
        return f'{self.__class__.__name__}(imię={self.imię!r}, pozycja={self.pozycja!r}, hand={self.hand()})'
    
    def store_card(self, karta: Karta|None):
        if karta is not None:
            self.append(karta)
        
    def hand(self)->list[str]:
        ''' lista kart w ręce grającego w kolejności rozdania'''
        return(self.get_labels(self))
    
    def hand_sorted(self)->list[str]:
        """Sortowanie domyślne: po kolorze, potem po wartości"""
        return self.hand_sorted_by_color()
    
    def hand_sorted_by_color(self)->list[str]:
        """Sortowanie po kolorze (♠️♥️♣️♦️), potem po wartości"""
        posortowane = sorted(
            self,
            key=lambda karta:(kolory.index(karta.kolor), wartości.index(karta.wartość)),
            reverse=False
        )
        return self.get_labels(posortowane)
    
    def hand_sorted_by_value(self)->list[str]:
        """Sortowanie po wartości (A,K,Q,W,...), potem po kolorze"""
        posortowane = sorted(
            self,
            key=lambda karta:(wartości.index(karta.wartość), kolory.index(karta.kolor)),
            reverse=False
        )
        return self.get_labels(posortowane)
    
    def hand_pretty(self, sort_by: str ='color')->str:
        """Beautifully formatted hand grouped by color"""
        result:list[str] = []
        if sort_by == 'color':
            sorted_cards = sorted(
                self,
                key=lambda k:(kolory.index(k.kolor), wartości.index(k.wartość))
            )
            # Grupuj karty po kolorach
            by_color: dict[str, list[str]] = {color: [] for color in kolory}
            for karta in sorted_cards:
                by_color[karta.kolor].append(karta.get_label())
            for color in kolory:
                if by_color[color]:
                    result.append(f"{color}: {' '.join(by_color[color])}")
        else:
            sorted_cards = sorted(
                self,
                key=lambda k:(wartości.index(k.wartość), kolory.index(k.kolor))
            )
            # Grupuj karty po wartościach
            by_value: dict[str, list[str]] = {value: [] for value in wartości}
            for karta in sorted_cards:
                by_value[karta.wartość].append(karta.get_label())
            for value in wartości:
                if by_value[value]:
                    result.append(f"{value}: {' '.join(by_value[value])}")
        
        return '\n'.join(result) if result else "Brak kart"
    
class Players(list[Gracz]):
    '''definicja grupy grających'''

    def __init__(self, *args: list[Gracz]|None, deck: Talia|None = None):
        super().__init__(*args)
        self.deck:Talia = deck if deck is not None else Talia()

    def __str__(self):
        return f'{[str(gracz) for gracz in self]}, deck={self.deck.__str__()}'
    
    def __repr__(self):
        return f'{self.__class__.__name__}({super().__repr__()}, deck={self.deck.__repr__()})'
    
    def deal(self): #rozdanie 13 kart każdemu z grających
        for _ in range(13):
            for gracz in self:
                gracz.store_card(self.deck.get_first_card())

In [23]:
# program testujący działanie klas i metod
if __name__ == "__main__" :
    # obsadzenie stołu i rozdanie kart
    the_table = Players(
        [Gracz(imię="Piotr",pozycja=pozycje[0]), 
        Gracz(imię="Agata", pozycja=pozycje[1]), 
        Gracz(imię="Kasia",pozycja=pozycje[2])], 
        deck=Talia())
    the_table.append(Gracz(imię="Karol", pozycja=pozycje[3])) # dodanie kolejnego gracza do stołu

    print(f'stół: {the_table}\n')

    the_table.deck.shuffle() # tasowanie talii
    the_table.deal() # rozdanie kart

    # ręce i talia po rozdaniu
    for gracz in the_table: # rozdane karty każdego gracza
        end = '\n' if gracz == the_table[-1] else ''
        print(f'rozdane: {gracz}{end}')

    for gracz in the_table: # posortowane karty każdego gracza
        end = '\n' if gracz == the_table[-1] else ''
        print(f'posortowane: {gracz}{end}')

    print(f'talia po rozdaniu: {the_table.deck}, pozostało: {the_table.deck.liczba_kart()}\n')  

stół: ['Piotr, N - []', 'Agata, E - []', 'Kasia, S - []', 'Karol, W - []'], deck=['A♠️', 'K♠️', 'Q♠️', 'W♠️', '10♠️', '9♠️', '8♠️', '7♠️', '6♠️', '5♠️', '4♠️', '3♠️', '2♠️', 'A♥️', 'K♥️', 'Q♥️', 'W♥️', '10♥️', '9♥️', '8♥️', '7♥️', '6♥️', '5♥️', '4♥️', '3♥️', '2♥️', 'A♣️', 'K♣️', 'Q♣️', 'W♣️', '10♣️', '9♣️', '8♣️', '7♣️', '6♣️', '5♣️', '4♣️', '3♣️', '2♣️', 'A♦️', 'K♦️', 'Q♦️', 'W♦️', '10♦️', '9♦️', '8♦️', '7♦️', '6♦️', '5♦️', '4♦️', '3♦️', '2♦️']

rozdane: Piotr, N - ['4♥️', 'A♥️', 'A♣️', '5♠️', '6♥️', '8♦️', '3♦️', 'W♠️', '10♣️', '8♣️', '7♦️', 'A♠️', 'W♥️']
rozdane: Agata, E - ['9♠️', '3♥️', '9♦️', '8♥️', 'Q♦️', '7♠️', '4♣️', 'A♦️', '6♦️', 'Q♠️', '9♥️', '2♦️', '2♥️']
rozdane: Kasia, S - ['K♠️', '6♣️', '6♠️', '4♠️', '7♥️', '2♣️', 'Q♣️', '4♦️', '3♠️', '5♦️', '10♥️', '10♠️', 'K♦️']
rozdane: Karol, W - ['K♥️', '7♣️', '5♥️', 'Q♥️', 'K♣️', '8♠️', '3♣️', '5♣️', 'W♣️', '10♦️', '2♠️', 'W♦️', '9♣️']

posortowane: Piotr, N - ['4♥️', 'A♥️', 'A♣️', '5♠️', '6♥️', '8♦️', '3♦️', 'W♠️', '10♣️', '8♣️', 

In [24]:
# Notebookowy interfejs do rozdawania i podglądu rąk
def show_table(table:Players):
    for gracz in table:
        print(f'{gracz.imię} ({gracz.pozycja}): {gracz.hand_sorted()}')

def create_and_deal(num_players:int=4, names:list[str]|None=None):
    if names is None:
        names = [f'Gracz{i+1}' for i in range(num_players)]
    players = Players([Gracz(imię=names[i] if i < len(names) else '', pozycja=pozycje[i]) for i in range(num_players)], deck=Talia())
    players.deck.shuffle()
    players.deal()
    return players

def run_cli():
    n = input('Liczba graczy (2-4, domyślnie 4): ' )
    try:
        n = int(n) if str(n).strip() != '' else 4
    except Exception:
        n = 4
    names:list[str]=[]
    for i in range(n):
        nm = input(f'Imię gracza {i+1} (enter — domyślne): ')
        names.append(nm.strip() if str(nm).strip() != '' else f'Gracz{i+1}')
    table = create_and_deal(n, names)
    print('\nRozegrano:')
    show_table(table)
    print(f'\nTalia pozostało: {table.deck.liczba_kart()}')

# Uruchom run_cli() aby rozpocząć interaktywnie.

In [25]:
import unittest

class TestKarta(unittest.TestCase):
    def test_karta_init_with_values(self):
        karta = Karta('A', '♠️')
        self.assertEqual(karta.wartość, 'A')
        self.assertEqual(karta.kolor, '♠️')
    
    def test_karta_init_empty(self):
        karta = Karta()
        self.assertEqual(karta.wartość, '')
        self.assertEqual(karta.kolor, '')
    
    def test_karta_get_label_normal(self):
        karta = Karta('K', '♥️')
        self.assertEqual(karta.get_label(), 'K♥️')
    
    def test_karta_get_label_empty(self):
        karta = Karta()
        self.assertEqual(karta.get_label(), '?')
    
    def test_karta_str(self):
        karta = Karta('10', '♣️')
        self.assertEqual(str(karta), '10♣️')
    
    def test_karta_repr(self):
        karta = Karta('Q', '♦️')
        repr_str = repr(karta)
        self.assertIn('Karta', repr_str)
        self.assertIn('Q', repr_str)

class TestTalia(unittest.TestCase):
    def test_talia_init_empty_list(self):
        talia = Talia()
        self.assertEqual(len(talia), 52)
    
    def test_talia_init_with_cards(self):
        karta1 = Karta('A', '♠️')
        karta2 = Karta('K', '♥️')
        talia = Talia([karta1, karta2])
        self.assertEqual(len(talia), 2)
    
    def test_talia_shuffle(self):
        talia1 = Talia()
        talia2 = Talia()
        talia1.shuffle()
        labels1 = talia1.w_stosie()
        labels2 = talia2.w_stosie()
        # bardzo mało prawdopodobne, że tasowana talia będzie identyczna jak nietasowana
        self.assertNotEqual(labels1, labels2)
    
    def test_talia_get_first_card(self):
        talia = Talia()
        first = talia.get_first_card()
        self.assertIsInstance(first, Karta)
        self.assertEqual(len(talia), 51)
    
    def test_talia_get_first_card_multiple(self):
        talia = Talia()
        for _ in range(5):
            card = talia.get_first_card()
            self.assertIsInstance(card, Karta)
        self.assertEqual(len(talia), 47)
    
    def test_talia_liczba_kart(self):
        talia = Talia()
        self.assertEqual(talia.liczba_kart(), 52)
        talia.get_first_card()
        self.assertEqual(talia.liczba_kart(), 51)

class TestGracz(unittest.TestCase):
    def test_gracz_init(self):
        gracz = Gracz(imię='Jan', pozycja='N')
        self.assertEqual(gracz.imię, 'Jan')
        self.assertEqual(gracz.pozycja, 'N')
        self.assertEqual(len(gracz), 0)
    
    def test_gracz_store_card(self):
        gracz = Gracz(imię='Maria', pozycja='E')
        karta = Karta('A', '♠️')
        gracz.store_card(karta)
        self.assertEqual(len(gracz), 1)
    
    def test_gracz_hand(self):
        gracz = Gracz(imię='Piotr', pozycja='S')
        gracz.store_card(Karta('K', '♥️'))
        gracz.store_card(Karta('Q', '♦️'))
        hand = gracz.hand()
        self.assertEqual(len(hand), 2)
        self.assertIn('K♥️', hand)
    
    def test_gracz_str_with_cards(self):
        gracz = Gracz(imię='Agata', pozycja='W')
        gracz.store_card(Karta('10', '♣️'))
        gracz_str = str(gracz)
        self.assertIn('Agata', gracz_str)
        self.assertIn('W', gracz_str)
    
    def test_gracz_str_without_name(self):
        gracz = Gracz(pozycja='N')
        gracz_str = str(gracz)
        self.assertIn('Brak imienia', gracz_str)

class TestPlayers(unittest.TestCase):
    def test_players_init(self):
        gracz1 = Gracz(imię='A', pozycja='N')
        gracz2 = Gracz(imię='B', pozycja='E')
        players = Players([gracz1, gracz2])
        self.assertEqual(len(players), 2)
        self.assertIsInstance(players.deck, Talia)
    
    def test_players_deal(self):
        gracze = [Gracz(imię=f'G{i}', pozycja=pozycje[i]) for i in range(4)]
        players = Players(gracze, deck=Talia())
        players.deal()
        for gracz in players:
            self.assertEqual(len(gracz), 13)
        self.assertEqual(players.deck.liczba_kart(), 0)
    
    def test_players_append(self):
        players = Players()
        gracz = Gracz(imię='Karol', pozycja='N')
        players.append(gracz)
        self.assertEqual(len(players), 1)

# Uruchom testy
suite = unittest.TestLoader().loadTestsFromModule(__import__(__name__))
runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)
print(f'\nRaport: {result.testsRun} testów, {len(result.failures)} błędów, {len(result.errors)} wyjątków')


test_gracz_hand (__main__.TestGracz.test_gracz_hand) ... ok
test_gracz_init (__main__.TestGracz.test_gracz_init) ... ok


test_gracz_store_card (__main__.TestGracz.test_gracz_store_card) ... ok
test_gracz_str_with_cards (__main__.TestGracz.test_gracz_str_with_cards) ... ok
test_gracz_str_without_name (__main__.TestGracz.test_gracz_str_without_name) ... ok
test_karta_get_label_empty (__main__.TestKarta.test_karta_get_label_empty) ... ok
test_karta_get_label_normal (__main__.TestKarta.test_karta_get_label_normal) ... ok
test_karta_init_empty (__main__.TestKarta.test_karta_init_empty) ... ok
test_karta_init_with_values (__main__.TestKarta.test_karta_init_with_values) ... ok
test_karta_repr (__main__.TestKarta.test_karta_repr) ... ok
test_karta_str (__main__.TestKarta.test_karta_str) ... ok
test_players_append (__main__.TestPlayers.test_players_append) ... ok
test_players_deal (__main__.TestPlayers.test_players_deal) ... ok
test_players_init (__main__.TestPlayers.test_players_init) ... ok
test_talia_get_first_card (__main__.TestTalia.test_talia_get_first_card) ... ok
test_talia_get_first_card_multiple (__main


Raport: 20 testów, 0 błędów, 0 wyjątków


In [26]:
# Demonstracja ulepszonego sortowania i formatowania rąk

print("=" * 70)
print("DEMO: Różne sposoby sortowania i wyświetlania rąk")
print("=" * 70)

# Utwórz pojedynczego gracza z losowymi kartami
demo_gracz = Gracz(imię='Demonstracyjny', pozycja='N')
deck = Talia()
deck.shuffle()
for _ in range(13): demo_gracz.store_card(deck.get_first_card())

print(f"\n1. Sortowanie po kolorach (domyślne):")
print(f"   hand_sorted_by_color(): {demo_gracz.hand_sorted_by_color()}")

print(f"\n2. Sortowanie po wartościach:")
print(f"   hand_sorted_by_value(): {demo_gracz.hand_sorted_by_value()}")

print(f"\n3. Ładny format pogrupowany po kolorach:")
print("   hand_pretty():")
for line in demo_gracz.hand_pretty().split('\n'):
    print(f"   {line}")

print(f"\n4. Ładny format pogrupowany po wartościach:")
print("   hand_pretty(sort_by='value'):")
for line in demo_gracz.hand_pretty(sort_by='value').split('\n'):
    print(f"   {line}")

print("\n" + "=" * 70)
print("Demo całego stołu z ulepszonymi wyświetleniami:")
print("=" * 70)

demo_table = create_and_deal(4, ['Piotr', 'Agata', 'Kasia', 'Karol'])
for gracz in demo_table:
    print(f"\n► {gracz.imię} ({gracz.pozycja}):")
    for line in gracz.hand_pretty().split('\n'):
        print(f"  {line}")


DEMO: Różne sposoby sortowania i wyświetlania rąk

1. Sortowanie po kolorach (domyślne):
   hand_sorted_by_color(): ['A♠️', '10♠️', '8♠️', '7♠️', '5♠️', 'Q♥️', '8♣️', '6♣️', '5♣️', '2♣️', 'A♦️', '10♦️', '6♦️']

2. Sortowanie po wartościach:
   hand_sorted_by_value(): ['A♠️', 'A♦️', 'Q♥️', '10♠️', '10♦️', '8♠️', '8♣️', '7♠️', '6♣️', '6♦️', '5♠️', '5♣️', '2♣️']

3. Ładny format pogrupowany po kolorach:
   hand_pretty():
   ♠️: A♠️ 10♠️ 8♠️ 7♠️ 5♠️
   ♥️: Q♥️
   ♣️: 8♣️ 6♣️ 5♣️ 2♣️
   ♦️: A♦️ 10♦️ 6♦️

4. Ładny format pogrupowany po wartościach:
   hand_pretty(sort_by='value'):
   A: A♠️ A♦️
   Q: Q♥️
   10: 10♠️ 10♦️
   8: 8♠️ 8♣️
   7: 7♠️
   6: 6♣️ 6♦️
   5: 5♠️ 5♣️
   2: 2♣️

Demo całego stołu z ulepszonymi wyświetleniami:

► Piotr (N):
  ♠️: A♠️ 4♠️
  ♥️: W♥️ 8♥️ 7♥️ 6♥️ 4♥️
  ♣️: 10♣️ 6♣️ 4♣️ 2♣️
  ♦️: Q♦️ 5♦️

► Agata (E):
  ♠️: K♠️ W♠️ 9♠️
  ♥️: Q♥️ 9♥️
  ♣️: K♣️ 5♣️
  ♦️: W♦️ 10♦️ 9♦️ 8♦️ 7♦️ 6♦️

► Kasia (S):
  ♠️: Q♠️ 7♠️
  ♥️: K♥️ 3♥️ 2♥️
  ♣️: A♣️ Q♣️ W♣️ 9♣️ 8♣️ 7♣️
  ♦️: K♦